In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import pandas as pd
import numpy
import os
import os.path 
import matplotlib.pyplot as plt
import matplotlib.colors
import copy
import errno
import glob
import time
import calendar
import json
import pickle
import random
import netCDF4
from numpy import savez_compressed
from optparse import OptionParser
from scipy.interpolate import (UnivariateSpline, RectBivariateSpline, RegularGridInterpolator)
from sklearn.metrics import confusion_matrix

In [29]:
def calc_mean_prob_df(id):
    csv_files_dir = '/data1/fog/fognn/Hamid/FogNetImport/SHAP/'
    folders = sorted(os.listdir(csv_files_dir))
    df = pd.read_csv(os.path.join(csv_files_dir, folders[id], 'VIS_Prob_TEST.csv'), index_col= 0)

    output = df.iloc[:, 0].mean()
    #print(f"{id}-{folders[id]}: {df.shape} |{output} ")
    return output


def SHAP_Scores():    
    Weights = [0.2, 0.2, 0.2, 0.2, 0.2, 
               0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 
               0.0333333,  0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 0.0333333, 
               0.05, 0.05, 0.05, 0.05, 0.05,
               0.2] 

    G1_idx = [0, 5, 6, 7, 8, 15, 16, 17, 18, 19, 20, 25, 26, 27, 28, 30]
    G1_weights = [Weights[index] for index in G1_idx]   
    G1_Sub = [(calc_mean_prob_df(0)  - calc_mean_prob_df(30)), 
              (calc_mean_prob_df(5)  - calc_mean_prob_df(1)), 
              (calc_mean_prob_df(6)  - calc_mean_prob_df(2)), 
              (calc_mean_prob_df(7)  - calc_mean_prob_df(3)), 
              (calc_mean_prob_df(8)  - calc_mean_prob_df(4)), 
              (calc_mean_prob_df(15) - calc_mean_prob_df(9)),  
              (calc_mean_prob_df(16) - calc_mean_prob_df(10)), 
              (calc_mean_prob_df(17) - calc_mean_prob_df(11)), 
              (calc_mean_prob_df(18) - calc_mean_prob_df(12)), 
              (calc_mean_prob_df(19) - calc_mean_prob_df(13)), 
              (calc_mean_prob_df(20) - calc_mean_prob_df(14)), 
              (calc_mean_prob_df(25) - calc_mean_prob_df(21)),
              (calc_mean_prob_df(26) - calc_mean_prob_df(22)), 
              (calc_mean_prob_df(27) - calc_mean_prob_df(23)), 
              (calc_mean_prob_df(28) - calc_mean_prob_df(24)), 
              (calc_mean_prob_df(30) - calc_mean_prob_df(29))]
    G1_score = sum([a*b for a, b in zip(G1_weights, G1_Sub)])
    
    G2_idx = [1, 5, 9, 10, 11, 15, 16, 17, 21, 22, 23, 25, 26, 27, 29, 30]
    G2_weights = [Weights[index] for index in G2_idx]
    G2_Sub = [(calc_mean_prob_df(1)  - calc_mean_prob_df(30)), 
              (calc_mean_prob_df(5)  - calc_mean_prob_df(0)), 
              (calc_mean_prob_df(9)  - calc_mean_prob_df(2)), 
              (calc_mean_prob_df(10) - calc_mean_prob_df(3)), 
              (calc_mean_prob_df(11) - calc_mean_prob_df(4)), 
              (calc_mean_prob_df(15) - calc_mean_prob_df(6)),  
              (calc_mean_prob_df(16) - calc_mean_prob_df(7)), 
              (calc_mean_prob_df(17) - calc_mean_prob_df(8)), 
              (calc_mean_prob_df(21) - calc_mean_prob_df(12)), 
              (calc_mean_prob_df(22) - calc_mean_prob_df(13)), 
              (calc_mean_prob_df(23) - calc_mean_prob_df(14)), 
              (calc_mean_prob_df(25) - calc_mean_prob_df(18)),
              (calc_mean_prob_df(26) - calc_mean_prob_df(19)), 
              (calc_mean_prob_df(27) - calc_mean_prob_df(20)), 
              (calc_mean_prob_df(29) - calc_mean_prob_df(24)), 
              (calc_mean_prob_df(30) - calc_mean_prob_df(28))]
    G2_score = sum([a*b for a, b in zip(G2_weights, G2_Sub)])
 
    G3_idx = [2, 6, 9, 12, 13, 15, 18, 19, 21, 22, 24, 25, 26, 28, 29, 30]
    G3_weights = [Weights[index] for index in G3_idx]
    G3_Sub = [(calc_mean_prob_df(2)  - calc_mean_prob_df(30)), 
              (calc_mean_prob_df(6)  - calc_mean_prob_df(0)), 
              (calc_mean_prob_df(9)  - calc_mean_prob_df(1)), 
              (calc_mean_prob_df(12) - calc_mean_prob_df(3)), 
              (calc_mean_prob_df(13) - calc_mean_prob_df(4)), 
              (calc_mean_prob_df(15) - calc_mean_prob_df(5)),  
              (calc_mean_prob_df(18) - calc_mean_prob_df(7)), 
              (calc_mean_prob_df(19) - calc_mean_prob_df(8)), 
              (calc_mean_prob_df(21) - calc_mean_prob_df(10)), 
              (calc_mean_prob_df(22) - calc_mean_prob_df(11)), 
              (calc_mean_prob_df(24) - calc_mean_prob_df(14)), 
              (calc_mean_prob_df(25) - calc_mean_prob_df(16)),
              (calc_mean_prob_df(26) - calc_mean_prob_df(17)), 
              (calc_mean_prob_df(28) - calc_mean_prob_df(20)), 
              (calc_mean_prob_df(29) - calc_mean_prob_df(23)), 
              (calc_mean_prob_df(30) - calc_mean_prob_df(27))]
    G3_score = sum([a*b for a, b in zip(G3_weights, G3_Sub)])
    
    G4_idx = [3, 7, 10, 12, 14, 16, 18, 20, 21, 23, 24, 26, 27, 28, 29, 30]
    G4_weights = [Weights[index] for index in G4_idx]
    G4_Sub = [(calc_mean_prob_df(3)  - calc_mean_prob_df(30)), 
              (calc_mean_prob_df(7)  - calc_mean_prob_df(0)), 
              (calc_mean_prob_df(10) - calc_mean_prob_df(1)), 
              (calc_mean_prob_df(12) - calc_mean_prob_df(2)), 
              (calc_mean_prob_df(14) - calc_mean_prob_df(4)), 
              (calc_mean_prob_df(16) - calc_mean_prob_df(5)),  
              (calc_mean_prob_df(18) - calc_mean_prob_df(6)), 
              (calc_mean_prob_df(20) - calc_mean_prob_df(8)), 
              (calc_mean_prob_df(21) - calc_mean_prob_df(9)), 
              (calc_mean_prob_df(23) - calc_mean_prob_df(11)), 
              (calc_mean_prob_df(24) - calc_mean_prob_df(13)), 
              (calc_mean_prob_df(25) - calc_mean_prob_df(15)),
              (calc_mean_prob_df(27) - calc_mean_prob_df(17)), 
              (calc_mean_prob_df(28) - calc_mean_prob_df(19)), 
              (calc_mean_prob_df(29) - calc_mean_prob_df(22)), 
              (calc_mean_prob_df(30) - calc_mean_prob_df(26))]

    G4_score = sum([a*b for a, b in zip(G4_weights, G4_Sub)])

    G5_idx     = [4, 8, 11, 13, 14, 17, 19, 20, 22, 23, 24, 26, 27, 28, 29, 30] 
    G5_weights = [Weights[index] for index in G5_idx]
    G5_Sub     = [(calc_mean_prob_df(4)  - calc_mean_prob_df(30)), 
                (calc_mean_prob_df(8)  - calc_mean_prob_df(0)), 
                (calc_mean_prob_df(11) - calc_mean_prob_df(1)), 
                (calc_mean_prob_df(13) - calc_mean_prob_df(2)), 
                (calc_mean_prob_df(14) - calc_mean_prob_df(3)), 
                (calc_mean_prob_df(17) - calc_mean_prob_df(5)),  
                (calc_mean_prob_df(19) - calc_mean_prob_df(6)), 
                (calc_mean_prob_df(20) - calc_mean_prob_df(7)), 
                (calc_mean_prob_df(22) - calc_mean_prob_df(9)), 
                (calc_mean_prob_df(23) - calc_mean_prob_df(10)), 
                (calc_mean_prob_df(24) - calc_mean_prob_df(12)), 
                (calc_mean_prob_df(26) - calc_mean_prob_df(15)),
                (calc_mean_prob_df(27) - calc_mean_prob_df(16)), 
                (calc_mean_prob_df(28) - calc_mean_prob_df(18)), 
                (calc_mean_prob_df(29) - calc_mean_prob_df(21)), 
                (calc_mean_prob_df(30) - calc_mean_prob_df(25))]

    G5_score = sum([a*b for a, b in zip(G5_weights, G5_Sub)])
    
    return G1_score, G2_score, G3_score, G4_score, G5_score

In [31]:
G1_score, G2_score, G3_score, G4_score, G5_score = SHAP_Scores()
print(f"SHAP Score of G1: {G1_score}")
print(f"SHAP Score of G2: {G2_score}")
print(f"SHAP Score of G3: {G3_score}")
print(f"SHAP Score of G4: {G4_score}")
print(f"SHAP Score of G5: {G5_score}")

SHAP Score of G1: -0.04745178710947809
SHAP Score of G2: -0.045873099200378915
SHAP Score of G3: 0.034714893776016255
SHAP Score of G4: 0.03041422302646109
SHAP Score of G5: 0.028195675482686696
